In [ ]:
import pandas as pd
import datetime as dt
import simulate
import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets import interact
from ipywidgets import fixed
import pathlib
print(pathlib.Path().absolute())


# Creating returns to simulate
spx = pd.read_csv('^GSPC.csv', index_col=0)
savings_year = pd.read_csv('investment_plan_year.csv', sep=';', index_col=0)
savings_year.index = pd.to_datetime(savings_year.index, format='%Y')
savings_month = (savings_year.resample('BMS').pad() / 12)['Earnings'].values

YEARLY_RF = 0.015
YEARLY_RM = 0.04  # Weighted average of margin rates

# --- Fixed parameters ----
investments = savings_month * 0.05

START = dt.date(2020, 1, 1)
END = dt.date(2070, 1, 31)
Market = simulate.Market(spx.iloc[-7500:, -2], START, END)

GAMMA = 2.5
SIGMA = Market.yearly.pct_change().std() ** 2
MR = (Market.yearly[-1] / Market.yearly[0]) ** (1 / len(Market.yearly)) - 1
COST = 0
# --- End fixed parameters ----


In [ ]:
import calculate_return

def plot_sims(y, sim_type, random_seeds, BEGINNING_SAVINGS = 9041,
                   YEARLY_INFL_ADJUSTMENT = 0.0, PAY_TAXES = True, YEARS = 60, GAMMA = 2.5,
                   YEARLY_RF = 0.02, YEARLY_MR = 0.04, COST = 0.002,
                   SIGMA = 0.02837, MR = 0.076):
    
    ports = calculate_return.fetch_returns(sim_type, random_seeds, BEGINNING_SAVINGS,
                   YEARLY_INFL_ADJUSTMENT, PAY_TAXES, YEARS, GAMMA,
                   YEARLY_RF, YEARLY_MR, COST,
                   SIGMA, MR)
    ports = ports.groupby(level=0).mean()
    fig, ax= plt.subplots(1, 1, figsize=(10,7))
    ax.plot(ports.loc[:,[*y]])
    ax.legend([*y])
    plt.show()


In [ ]:
#plot_sims(['tv_u'], 'garch', range(100))
from ipywidgets import interact
from ipywidgets import fixed
import matplotlib.pyplot as plt


interactive_plot = interact(plot_sims, y=fixed(['tv_u', '100', '9050']), sim_type = ['garch', 't', 'norm', 'draw'],
                                                               random_seeds = fixed(range(15)), 
                           BEGINNING_SAVINGS = range(6000, 20000, 1000),
                   YEARLY_INFL_ADJUSTMENT = (0, 0.05, 0.01), PAY_TAXES = True, YEARS = (20, 60, 10), GAMMA = (1, 2.5, 0.1),
                   YEARLY_RF = (0, 0.04, 0.005), YEARLY_MR = (0.03, 0.08, 0.005), COST=(0, 0.01, 0.001),
                   SIGMA = fixed(0.02837), MR = fixed(0.076))
interactive_plot